In [15]:
################################################################################
## Code adapted from demo_caiman_cnmf_3D as imported from github 21/11/2018
## https://github.com/flatironinstitute/CaImAn
################################################################################

import cde_cell_functions as cc

# Import relevant packages
#===============================================================================
import imp
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
import shutil
from scipy.ndimage.filters import gaussian_filter
import scipy.sparse
import sys
import re
from skimage.external.tifffile import imread
from skimage import io
import warnings 
from IPython.display import clear_output
import copy
from importlib import reload
from PIL import Image
import datetime

# Caiman setup
#-------------------------------------------------------------------------------
import caiman as cm
import caiman.source_extraction.cnmf as cnmf
from caiman.utils.visualization import nb_view_patches3d
from caiman.source_extraction.cnmf import params as params
from caiman.components_evaluation import evaluate_components, estimate_components_quality_auto
from caiman.motion_correction import MotionCorrect
from caiman.cluster import setup_cluster
from caiman.paths import caiman_datadir
from caiman.utils.visualization import inspect_correlation_pnr

# Jupyter specific autoreloading for external functions (in case changes are made)
# %load_ext autoreload
# %autoreload

In [16]:
# Housekeeping
#===============================================================================
# Module flags

display_movie   = False      # play movie of tifs that are loaded
save_results    = False      # flag to save results or not

# Define folder locations
#-------------------------------------------------------------------------------
reload(cc)
Fbase     = '/Users/roschkoenig/Dropbox/Research/1812 Critical Dynamics Epilepsy'
Fsave     = '/Users/roschkoenig/Dropbox/Research/1812 Critical Dynamics Epilepsy Data'
Fscripts  = Fbase + os.sep + '03 - Cell detection' 
Fdata     = '/Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy'
Zfish     = cc.cde_cell_fishspec(Fdata, 'RM')

In [19]:
Fish = Zfish[1:]
Fish[0]["Name"]

'ZFRR012_01'

# Split images up into planes

In [20]:
for z in Fish:
    print('----------------------------------------------------------------------------')
    print('Currently working on ' + z["Name"])
    print('----------------------------------------------------------------------------')
    cc.cde_cell_planesave(Fdata, z, mxpf = 7000)

----------------------------------------------------------------------------
Currently working on ZFRR012_01
----------------------------------------------------------------------------
I found 2 Conditions
Condition BL
> There are 9 Planes
> Processing plane 1
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-14 15:41:18| Saving interim file
>  2019-04-14 15:42:40| Saving interim file
> Processing plane 2
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-14 18:07:22| Saving interim file
>  2019-04-14 18:08:40| Saving interim file
> Processing plane 3
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-14 20:21:55| Saving interim file
>  2019-04-14 20:23:16| Saving interim file
> Processing plane 4
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-14 22:35:22| Saving interim file
>  2019-04-14 22:36:43| Saving interim file
> Processing plane 5
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-15 00:49:07| Saving interim file
>  2019-04-15 00:50:26| Saving interim file
> Processing plane 

/Users/roschkoenig/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: /Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy/PL_ZFRR012_01/PTZ/ZFRR012_01_s02_P_PL00.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


> Processing plane 2
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-15 17:17:24| Saving interim file
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-15 19:39:17| Saving interim file
>  2019-04-15 19:41:03| Saving interim file


/Users/roschkoenig/.local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: /Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy/PL_ZFRR012_01/PTZ/ZFRR012_01_s02_P_PL01.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


> Processing plane 3
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-15 21:58:15| Saving interim file
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 00:16:55| Saving interim file
>  2019-04-16 00:18:42| Saving interim file
> Processing plane 4
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 02:38:50| Saving interim file
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 05:00:26| Saving interim file
>  2019-04-16 05:02:14| Saving interim file
> Processing plane 5
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 07:18:06| Saving interim file
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 09:30:09| Saving interim file
>  2019-04-16 09:31:54| Saving interim file
> Processing plane 6
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-04-16 11:46:11| Saving interim file
>  >  >  >  >  >  >  >  >  >  >  >  >  >  

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy/RM_ZFRR012_01/PTZ/13658_P.tif'

2019-03-13 17:00:47


In [9]:
z["Cond"][1]

{'Name': 'PTZ',
 'Path': '/Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy/RM_ZFRR008_02/PTZ',
 'Plane': [],
 'Tifs': [],
 'Tpaths': []}

# Run actual cell segmentation

In [8]:
imp.reload(cc),  
Pfish = cc.cde_cell_fishspec(Fdata, 'PL')
f     = 3

In [261]:
imp.reload(cc)

<module 'cde_cell_functions' from '/Users/roschkoenig/Dropbox/Research/1812 Critical Dynamics Epilepsy/03 - Cell detection/cde_cell_functions.py'>

In [ ]:
for c in range(len(Pfish[f]["Cond"])):
    planes = Pfish[f]["Cond"][c]["Plane"]
    Estimates = []
    for p in range(len(planes)):
        fname = planes[p]["Tpaths"]
        fr = 4                                                              # frame rate (Hz)
        decay_time = 0.5                                                    # approximate length of transient event in seconds
        gSig = (4,4)                                                        # expected half size of neurons
        p = 1                                                               # order of AR indicator dynamics
        min_SNR = 1                                                         # minimum SNR for accepting new components
        rval_thr = 0.90                                                     # correlation threshold for new component inclusion
        ds_factor = 1                                                       # spatial downsampling factor (increases speed but may lose some fine structure)
        gnb = 2                                                             # number of background components
        gSig = tuple(np.ceil(np.array(gSig)/ds_factor).astype('int'))       # recompute gSig if downsampling is involved
        mot_corr = True                                                     # flag for online motion correction 
        pw_rigid = False                                                    # flag for pw-rigid motion correction (slower but potentially more accurate)
        max_shifts_online = np.ceil(10./ds_factor).astype('int')            # maximum allowed shift during motion correction
        sniper_mode = True                                                  # flag using a CNN to detect new neurons (o/w space correlation is used)
        init_batch = 200                                                    # number of frames for initialization (presumably from the first file)
        expected_comps = 500                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
        dist_shape_update = True                                            # flag for updating shapes in a distributed way
        min_num_trial = 10                                                  # number of candidate components per frame     
        K = 2                                                               # initial number of components
        epochs = 2                                                          # number of passes over the data
        show_movie = False                                                  # show the movie with the results as the data gets processed

        params_dict = {'fnames': fname,
                       'fr': fr,
                       'decay_time': decay_time,
                       'gSig': gSig,
                       'p': p,
                       'min_SNR': min_SNR,
                       'rval_thr': rval_thr,
                       'ds_factor': ds_factor,
                       'nb': gnb,
                       'motion_correct': mot_corr,
                       'init_batch': init_batch,
                       'init_method': 'bare',
                       'normalize': True,
                       'expected_comps': expected_comps,
                       'sniper_mode': sniper_mode,
                       'dist_shape_update' : dist_shape_update,
                       'min_num_trial': min_num_trial,
                       'K': K,
                       'epochs': epochs,
                       'max_shifts_online': max_shifts_online,
                       'pw_rigid': pw_rigid,
                       'show_movie': show_movie}
        opts = cnmf.params.CNMFParams(params_dict=params_dict)
        clear_output()
        print('-----------------------------------------------------------------------')
        print('Currently processing condition ' + Pfish[f]["Cond"][c]["Name"])
        print('> Plane ' + str(p) + ' of ' + str(len(planes)))
        print('-----------------------------------------------------------------------')
        cmn  = cnmf.online_cnmf.OnACID(params=opts)
        cmn.fit_online()
        Estimates.append({'Spatial':cnm.estimates.A,'Temporal':cnm.estimates.C,'Background':cnm.estimates.b})
    Pfish[f]["Cond"][c].update({"CMN":Estimates})

-----------------------------------------------------------------------
Currently processing condition PTZ
> Plane 2 of 9
-----------------------------------------------------------------------
Size frame:(298, 451)
Noise Normalization
Roi Extraction...
Greedy initialization of spatial and temporal components using spatial Gaussian filtering
USING TOTAL SUM FOR INITIALIZATION....
(Hals) Refining Components...
Expecting 500 components
Now processing file /Volumes/MARIANNE/1812 Critical Dynamics in Epilepsy/PL_ZFRR004_02/PTZ/ZFRR004_02_s00_P_PL03.tif
Epoch: 1. 200 frames have beeen processed in total. 0 new components were added. Total # of components is 2
Epoch: 1. 300 frames have beeen processed in total. 80 new components were added. Total # of components is 82
Epoch: 1. 400 frames have beeen processed in total. 39 new components were added. Total # of components is 121
Epoch: 1. 500 frames have beeen processed in total. 38 new components were added. Total # of components is 159
Epoch

Increasing number of expected components to:700
Epoch: 1. 1800 frames have beeen processed in total. 22 new components were added. Total # of components is 509
Epoch: 1. 1900 frames have beeen processed in total. 21 new components were added. Total # of components is 530
Epoch: 1. 2000 frames have beeen processed in total. 13 new components were added. Total # of components is 543
Epoch: 1. 2100 frames have beeen processed in total. 13 new components were added. Total # of components is 556
Epoch: 1. 2200 frames have beeen processed in total. 12 new components were added. Total # of components is 568
Epoch: 1. 2300 frames have beeen processed in total. 17 new components were added. Total # of components is 585
Epoch: 1. 2400 frames have beeen processed in total. 13 new components were added. Total # of components is 598
Epoch: 1. 2500 frames have beeen processed in total. 21 new components were added. Total # of components is 619
Epoch: 1. 2600 frames have beeen processed in total. 13 

Increasing number of expected components to:900
Epoch: 1. 5100 frames have beeen processed in total. 12 new components were added. Total # of components is 702
Epoch: 1. 5200 frames have beeen processed in total. 20 new components were added. Total # of components is 722
Epoch: 1. 5300 frames have beeen processed in total. 7 new components were added. Total # of components is 729
Epoch: 1. 5400 frames have beeen processed in total. 11 new components were added. Total # of components is 740
Epoch: 1. 5500 frames have beeen processed in total. 13 new components were added. Total # of components is 753
Epoch: 1. 5600 frames have beeen processed in total. 22 new components were added. Total # of components is 775
Epoch: 1. 5700 frames have beeen processed in total. 7 new components were added. Total # of components is 782
Epoch: 1. 5800 frames have beeen processed in total. 31 new components were added. Total # of components is 813
Epoch: 1. 5900 frames have beeen processed in total. 13 ne

Increasing number of expected components to:1100
Epoch: 1. 6400 frames have beeen processed in total. 5 new components were added. Total # of components is 899
Epoch: 1. 6500 frames have beeen processed in total. 4 new components were added. Total # of components is 903
Epoch: 1. 6600 frames have beeen processed in total. 4 new components were added. Total # of components is 907
Epoch: 1. 6700 frames have beeen processed in total. 0 new components were added. Total # of components is 907
Epoch: 1. 6800 frames have beeen processed in total. 0 new components were added. Total # of components is 907
Epoch: 1. 6900 frames have beeen processed in total. 0 new components were added. Total # of components is 907
Epoch: 1. 7000 frames have beeen processed in total. 0 new components were added. Total # of components is 907
Epoch: 1. 7100 frames have beeen processed in total. 0 new components were added. Total # of components is 907
Epoch: 1. 7200 frames have beeen processed in total. 0 new comp

In [253]:
Estimates = Secure_Estimates[0:9]
Pfish[f]["Cond"][0].update({"CMN":Estimates})

In [301]:
# Save everyhting into folder
#---------------------------------------------------------------------------------
Fcmn = Fsave + os.sep + 'Analysis' + os.sep + 'CMN' + os.sep + Pfish[f]["Name"]
if not os.path.exists(Fcmn): os.makedirs(Fcmn)
    
for c in range(len(Pfish[f]["Cond"])):
    Fccond = Fcmn + os.sep + Pfish[f]["Cond"][c]["Name"]
    if not os.path.exists(Fccond):
        os.makedirs(Fccond)
    for p in range(len(Pfish[f]["Cond"][c]["CMN"])):
        scipy.io.savemat(Fccond + os.sep + Pfish[f]["Name"] + '_P' + str(p).zfill(2), Pfish[f]["Cond"][c]["CMN"][p])